<a href="https://colab.research.google.com/github/BoWarburton/blnqr/blob/master/active_choice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Active Choice


### Introduction
Active Choice is a low-tech reinforcement learning model.
In this example it is tasked with finding the best move in a simple game of 'Rock Paper Scissors Lizard Spock` One model will be given 2 of the three strategies and the other will be given 3 of them.

![Image](https://cdn.instructables.com/FIU/AIWE/I7Q0TCUT/FIUAIWEI7Q0TCUT.LARGE.jpg?auto=webp&fit=bounds)

### Explanation
... Todo

### Summary
... Todo


In [0]:
!pip install Fortuna

     |████████████████████████████████| 194kB 2.7MB/s 
  Created wheel for Fortuna: filename=Fortuna-3.17.6-cp36-cp36m-linux_x86_64.whl size=528699 sha256=11e497905e9df0988dec20e3b905a9e621be72a5fdcd69bcf0d037f3c6766872
  Stored in directory: /root/.cache/pip/wheels/f0/9c/d2/4c0b30bf48071e20dd4ca6da7c3a031227264db650fa0066c2
Successfully built Fortuna


In [0]:
from Fortuna import RelativeWeightedChoice


class ActiveChoice:

    def __init__(self, name, data, start=50, lo=1, hi=100):
        self.name = name
        self.raw_data = {k: start for k in data}
        self.start = start
        self.lo = lo
        self.hi = hi
        self.data = self.build()
        self.wins = 0.0
        self.losses = 0.0

    def __call__(self):
        return self.data()

    def build(self):
        return RelativeWeightedChoice(
            zip(self.raw_data.values(), self.raw_data.keys()))

    def winner(self, result):
        self.wins += 1
        if self.raw_data[result] < self.hi:
            self.raw_data[result] += 1
            self.data = self.build()

    def looser(self, result):
        self.losses += 1
        if self.raw_data[result] > self.lo:
            self.raw_data[result] -= 1
            self.data = self.build()

    def win_ratio(self):
        total_games = self.wins + self.losses
        return self.wins / total_games if total_games else total_games

    def __str__(self):
        max_weight = max(self.raw_data.values())
        output = (
            f"{self.name}: {self.raw_data}",
            f"Best strategies: " + ", ".join(
                k for k, v in self.raw_data.items() if v == max_weight),
            f"Win Rate: {self.win_ratio():.2%}",
            ""
        ) if self.win_ratio() else (f"{self.name}: {self.raw_data}", "")
        return "\n".join(output)

In [0]:
def run_game(team_a, team_b, cycles=10000):
    game_rules = {
        "rock": {
            "rock": 0, "paper": -1, "scissors": 1, "lizard": 1, "spock": -1},
        "paper": {
            "rock": 1, "paper": 0, "scissors": -1, "lizard": -1, "spock": 1},
        "scissors": {
            "rock": -1, "paper": 1, "scissors": 0, "lizard": 1, "spock": -1},
        "lizard": {
            "rock": -1, "paper": 1, "scissors": -1, "lizard": 0, "spock": 1},
        "spock": {
            "rock": 1, "paper": -1, "scissors": 1, "lizard": -1, "spock": 0},
    }

    def game():
        a = team_a()
        b = team_b()
        this_game = game_rules[a][b]
        if this_game > 0:
            team_a.winner(a)
            team_b.looser(b)
        elif this_game < 0:
            team_a.looser(a)
            team_b.winner(b)
        else:
            team_a.looser(a)
            team_b.looser(b)

    for _ in range(cycles):
        game()

In [0]:
team_1 = ActiveChoice("Team 1", ("lizard", "scissors"))
team_2 = ActiveChoice("Team 2", ("rock", "paper", "spock"))

run_game(team_1, team_2)
print(team_1)
print(team_2)

Team 1: {'lizard': 1, 'scissors': 1}
Best strategies: lizard, scissors
Win Rate: 4.04%

Team 2: {'rock': 100, 'paper': 1, 'spock': 5}
Best strategies: rock
Win Rate: 95.96%

